In [ ]:
library(ppcor)
#install.packages('lmPerm')
library(lmPerm)
library(psych)
library(car)
library(ggplot2)
library(lme4)
library(mediation)
library(caret)
library(gbm)
library(party)
library(Metrics)
#install.packages("wesanderson")
library(wesanderson)
library(gtools)
library(interactions)
#install.packages('ggstance')
library(ggstance)
#install.packages('MatchIt')
library(MatchIt)
library(dplyr)
library(ukbtools)
library(mediation)
library(lavaan)
library(mice)

In [ ]:
my_ukb_data <- ukb_df("ukb38276", path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
load("/home/bic/fmorys/Projects/UKBB_all/di.Rda")
colnames(di)=c('eid',colnames(di)[2:length(di)])
ukb_area <- ukb_df('ukb40807', path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
glob2rx("area_of*_2_*") # Create a regexp to use in grep
ukbb_area=(ukb_area[c(1, grep('^area_of.*_2_',colnames(ukb_area)))])

### Exclude outliers in the imaging dataset (n=22k)
di$bipolar_dis <- NA
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type I (Mania)"] <- 1
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type II (Hypomania)"] <- 1

#Personality Disorders
di$personality_dis <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$personality_dis[di[i]=="A personality disorder"] <- 1
}

#Mental health issues
di$depression <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
#di$schizophrenia[di[i]=="Schizophrenia"] <- 1
di$depression[di[i]=="Depression"] <- 1
#di$schizophrenia[di[i]=="Mania, hypomania, bipolar or manic-depression"] <- 1
#di$schizophrenia[di[i]=="Bulimia nervosa"] <- 1
#di$schizophrenia[di[i]=="Anorexia nervosa"] <- 1
#di$schizophrenia[di[i]=="Attention deficit or attention deficit and hyperactivity disorder (ADD/ADHD)"] <- 1
#di$schizophrenia[di[i]=="Panic attacks"] <- 1
}


# Exclude participants who have diabetes
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('diabetes_diagnosed_by_doctor', 
                          colnames(di)))) { #Columns with vascular diseases
di$diabetes[di[i] =="Yes"] <- 1
}

# Exclude participants who had a heart attack, angina or stroke
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('vascularheart_problems_diagnosed_by_doctor', 
                          colnames(di)))) { #Columns with vascular diseases
di$vascular_heart_diagnoses[di[i] =="High blood pressure"] <- 1
}



#Dementia
for (i in as.numeric(grep('noncancer_illness_code_selfreported', 
                          colnames(di)))) { #Columns with noncancer illness for all timepoints
di$neurological_disorder[di[i] == "1263"] <- 1 #Dementia or Alzheimer's
di$neurological_disorder[di[i] == "1262"] <- 1 #Parkinson Disease
di$neurological_disorder[di[i] == "1258"] <- 1 #Chronic degenerative neurological
di$neurological_disorder[di[i] == "1256"] <- 1 #Guillain Barré
di$neurological_disorder[di[i] == "1261"] <- 1 #Multiple slerosis
di$neurological_disorder[di[i] == "1297"] <- 1 #Other demyenilating disease
di$neurological_disorder[di[i] == "1081"] <- 1 #Stroke
di$neurological_disorder[di[i] == "1032"] <- 1 #Brain cancer
di$neurological_disorder[di[i] == "1491"] <- 1 #Brain hemorrhage
di$neurological_disorder[di[i] == "1245"] <- 1 #Brain/ intracranial abcess
di$neurological_disorder[di[i] == "1425"] <- 1 #Cerebral aneurism
di$neurological_disorder[di[i] == "1433"] <- 1 #Cerebral palsy
di$neurological_disorder[di[i] == "1256"] <- 1 #Encephalitis
di$neurological_disorder[di[i] == "1264"] <- 1 #Epilepsy
di$neurological_disorder[di[i] == "1266"] <- 1 #Head injury
di$neurological_disorder[di[i] == "1244"] <- 1 #Infections of the nervous system
di$neurological_disorder[di[i] == "1583"] <- 1 #Ischaemic stroke
di$neurological_disorder[di[i] == "1031"] <- 1 #Meningeal cancer
di$neurological_disorder[di[i] == "1659"] <- 1 #Meningioma (benign)
di$neurological_disorder[di[i] == "1247"] <- 1 #Meningitis
di$neurological_disorder[di[i] == "1259"] <- 1 #Motor neuron disease
di$neurological_disorder[di[i] == "1240"] <- 1 #Neurological injury/trauma
di$neurological_disorder[di[i] == "1524"] <- 1 #Spina bifida
di$neurological_disorder[di[i] == "1083"] <- 1 #Subdural hematoma
di$neurological_disorder[di[i] == "1086"] <- 1 #Subarachnoid haemorrage
di$neurological_disorder[di[i] == "1082"] <- 1 #Transient ischaemic attack
}

#If BMI < 18.5 (Re-do this, cause it's not usually done)
#di$underweight1[di$body_mass_index_bmi_f21001_0_0 < 18.5] <- 1
#di$underweight2[di$body_mass_index_bmi_f21001_1_0 < 18.5] <- 1
di$underweight3[di$body_mass_index_bmi_f21001_2_0 < 18.5] <- 1

#Exclude participants if no neuroimaging data, bipolar_dis == 1; personality_dis == 1; schizophrenia == 1; vascular_heart_diagnoses ==1

di$excluded <- NA
##di$excluded[di$bipolar_dis ==1] <- 1
#di$excluded[di$personality_dis == 1] <- 1
#di$excluded[di$schizophrenia == 1] <- 1
#di$excluded[di$vascular_heart_diagnoses ==1] <- 1
#di$excluded[di$underweight1 ==1] <- 1
#di$excluded[di$underweight2 ==1] <- 1
#di$excluded[di$underweight3 == 1] <- 1
di$excluded[di$neurological_disorder ==1] <- 1
#di$excluded[di$diabetes ==1] <- 1

di$included<-car::recode(di$excluded, "1='excluded'; else='included'")


#Select subjects if included ==1
di_excluded <- subset(di, included=="included")

ukbb_all=merge(my_ukb_data, di_excluded, by.x='eid')
ukbb_all=merge(ukbb_all, ukbb_area, by='eid',all.x=TRUE)

nrow(ukbb_all)

ukbb_all$WMH_logscaled=log(ukbb_all$total_volume_of_white_matter_hyperintensities_from_t1_and_t2_flair_images_f25781_2_0/
                    ukbb_all$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0)
ukbb_all$logBMI3=log(ukbb_all$body_mass_index_bmi_f21001_0_0)
ukbb_all$WHR=ukbb_all$waist_circumference_f48_0_0/ukbb_all$hip_circumference_f49_0_0

## Merge with white matter integrity variables
ukb_WM <- ukb_df('ukb35375', path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
ukbb_all=merge(ukbb_all,ukb_WM,by='eid',all.x=TRUE)

#Remove outliers
variables_for_outliers=c(5095,5096, 5094,2003) # BMI< WHR< WMH< BFP, 

for (j in variables_for_outliers) { # 
    Q3=as.numeric(quantile(ukbb_all[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(ukbb_all[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    ukbb_all[j][ukbb_all[j]<lower]=NA
    ukbb_all[j][ukbb_all[j]>upper]=NA
}


ukbb_all$Hypertension=0
ukbb_all$Hypertension[ukbb_all$vascular_heart_diagnoses=='1']=1
ukbb_all$Diabetes=0
ukbb_all$Diabetes[ukbb_all$diabetes=='1']=1
ukbb_all$Depression=0
ukbb_all$Depression[ukbb_all$depression=='1']=1


PA=read.table('/dagher/dagher11/filip/OBAL/alcohol.csv',sep=',',header=T)
ukbb_all=merge(ukbb_all,PA,by='eid',all.x=T)
ukbb_all$Sex=ukbb_all$sex_f31_0_0

## Impute missing confouners with MICE
to_impute=c(447, 5554, 5560, 5559, 1732, 242, 1126)
imputed_data=mice(ukbb_all[to_impute],m=1, maxit = 50, method = 'pmm', seed = 500)
ukbb_all[to_impute] <- complete(imputed_data,1)

ukbb_all=ukbb_all[complete.cases(ukbb_all[,c(2003, 5094:5096, 1897, 447, 5554, 5560, 5559, 1732, 242)]),] # BMI, WMH, WHR, BFP + all confounders - used everywhere so might exclude as well here already
nrow(ukbb_all)

In [ ]:
UKB_Blood <- ukb_df("ukb41240", path = "/dagher/dagher11/filip/OBAL/UKBB_data/")

In [ ]:
ukbb_blood=merge(ukbb_all,UKB_Blood, by='eid', all.x=TRUE)

In [ ]:
ukbb_blood$SBP=(rowMeans(cbind(ukbb_blood$systolic_blood_pressure_automated_reading_f4080_0_0,
                           ukbb_blood$systolic_blood_pressure_automated_reading_f4080_0_1)))

ukbb_blood$DBP=(rowMeans(cbind(ukbb_blood$diastolic_blood_pressure_automated_reading_f4079_0_0.x,
                           ukbb_blood$diastolic_blood_pressure_automated_reading_f4079_0_1.x)))

In [ ]:
variables_for_outliers=c(6292, 6360, 6514, 6334, 6348, 6572, 6571) # 6284 6352 # CRP TG HDL Glucose HbA1c SBP DBP

for (j in variables_for_outliers) { # 
    Q3=as.numeric(quantile(ukbb_blood[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(ukbb_blood[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    ukbb_blood[j][ukbb_blood[j]<lower]=NA
    ukbb_blood[j][ukbb_blood[j]>upper]=NA
}

In [ ]:
UKBMED=data.frame(scale(ukbb_blood$logBMI3),ukbb_blood$WMH_logscaled,scale(log(ukbb_blood$creactive_protein_f30710_0_0)),
                 ukbb_blood$Hypertension,ukbb_blood$Diabetes, scale(ukbb_blood$WHR), scale(log(ukbb_blood$triglycerides_f30870_0_0)),
                 scale(log(ukbb_blood$hdl_cholesterol_f30760_0_0)),as.numeric(ukbb_blood$average_total_household_income_before_tax_f738_2_0),
                                scale(ukbb_blood$body_fat_percentage_f23099_0_0), 
                  scale(log(ukbb_blood$glucose_f30740_0_0)),
                  scale(log(ukbb_blood$glycated_haemoglobin_hba1c_f30750_0_0)),
                  scale(ukbb_blood$SBP),
                  scale(ukbb_blood$DBP),
                  ukbb_blood$Depression, 
                       as.numeric(ukbb_blood$frequency_of_drinking_alcohol_20414.0.0),
                                  ukbb_blood$number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0 ,
                                  as.numeric(ukbb_blood$smoking_status_f20116_2_0), 
                        as.numeric(ukbb_blood$qualifications_f6138_2_0) ,
                                  ukbb_blood$townsend_deprivation_index_at_recruitment_f189_0_0,
                 as.numeric(ukbb_blood$sex_f31_0_0),ukbb_blood$age_when_attended_assessment_centre_f21003_2_0)

colnames(UKBMED)=c('logBMI3','WMH','CRP','Hypertension','Diabetes','WHR','TG','HDL','Income',
                  'BFP','Glucose','HbA1c','SBP','DBP','Depression','Alcohol','PA','Smoking','Education',
                   'TDI','Sex','Age')

UKBMED=na.omit(UKBMED)

In [ ]:
nrow(UKBMED)

In [ ]:
for (i in 1:14)
    
    UKBMED[[i]]=resid(lm(UKBMED[[i]]~
                        Depression + Alcohol + PA + Smoking + Education +
                        TDI + Sex + Age, data=UKBMED))

## Parallel mediations models

## BMI

In [ ]:
model <- '
# direct effect
             WMH ~ c*logBMI3 

             CRP ~ a1*logBMI3 
             Hypertension ~ a2*logBMI3 
             Diabetes ~ a3*logBMI3 
             HDL ~ a4*logBMI3 
             TG ~ a5*logBMI3 
             Glucose ~ a6*logBMI3 
             HbA1c ~ a7*logBMI3 
             DBP ~ a8*logBMI3 
             SBP ~ a9*logBMI3 
             WMH ~ b1*CRP + b2*Hypertension + b3*Diabetes + b4*HDL + b5*TG + b6*Glucose + b7*HbA1c + b8*DBP + b9*SBP
# indirect effect (a*b)
             a1b1 := a1*b1
             a2b2 := a2*b2
             a3b3 := a3*b3
             a4b4 := a4*b4
             a5b5 := a5*b5
             a6b6 := a6*b6
             a7b7 := a7*b7
             a8b8 := a8*b8
             a9b9 := a9*b9
# total effect
             total := c + a1b1 + a2b2 + a3b3 + a4b4 + a5b5 + a6b6 + a7b7 + a8b8 + a9b9

         '
fit <- sem(model, data = UKBMED)
sumfit=summary(fit)

In [ ]:
options(scipen = 999)

BMIMed=data.frame('var'=c('CRP','HT','DM','HDL','TG','Glc','HbA1c','DBP','SBP'),
                  'pval'=p.adjust(sumfit$PE[31:39,]$pvalue, method='BH'),
                  'est'=sumfit$PE[31:39,]$est,
                  'prop'=sumfit$PE[31:39,]$est/sumfit$PE[40,]$est
                 )

BMIMed

## WHR

In [ ]:
model <- '
# direct effect
             WMH ~ c*WHR 

             CRP ~ a1*WHR 
             Hypertension ~ a2*WHR 
             Diabetes ~ a3*WHR 
             HDL ~ a4*WHR 
             TG ~ a5*WHR 
             Glucose ~ a6*WHR 
             HbA1c ~ a7*WHR 
             DBP ~ a8*WHR 
             SBP ~ a9*WHR 
             WMH ~ b1*CRP + b2*Hypertension + b3*Diabetes + b4*HDL + b5*TG + b6*Glucose + b7*HbA1c + b8*DBP + b9*SBP
# indirect effect (a*b)
             a1b1 := a1*b1
             a2b2 := a2*b2
             a3b3 := a3*b3
             a4b4 := a4*b4
             a5b5 := a5*b5
             a6b6 := a6*b6
             a7b7 := a7*b7
             a8b8 := a8*b8
             a9b9 := a9*b9
# total effect
             total := c + a1b1 + a2b2 + a3b3 + a4b4 + a5b5 + a6b6 + a7b7 + a8b8 + a9b9

         '
fit <- sem(model, data = UKBMED)
sumfit=summary(fit)

In [ ]:
options(scipen = 999)

WHRMed=data.frame('var'=c('CRP','HT','DM','HDL','TG','Glc','HbA1c','DBP','SBP'),
                  'pval'=p.adjust(sumfit$PE[31:39,]$pvalue, method='BH'),
                  'est'=sumfit$PE[31:39,]$est,
                  'prop'=sumfit$PE[31:39,]$est/sumfit$PE[40,]$est
                 )

WHRMed

In [ ]:
model <- '
# direct effect
             WMH ~ c*BFP 

             CRP ~ a1*BFP 
             Hypertension ~ a2*BFP 
             Diabetes ~ a3*BFP 
             HDL ~ a4*BFP 
             TG ~ a5*BFP 
             Glucose ~ a6*BFP 
             HbA1c ~ a7*BFP 
             DBP ~ a8*BFP 
             SBP ~ a9*BFP 
             WMH ~ b1*CRP + b2*Hypertension + b3*Diabetes + b4*HDL + b5*TG + b6*Glucose + b7*HbA1c + b8*DBP + b9*SBP
# indirect effect (a*b)
             a1b1 := a1*b1
             a2b2 := a2*b2
             a3b3 := a3*b3
             a4b4 := a4*b4
             a5b5 := a5*b5
             a6b6 := a6*b6
             a7b7 := a7*b7
             a8b8 := a8*b8
             a9b9 := a9*b9
# total effect
             total := c + a1b1 + a2b2 + a3b3 + a4b4 + a5b5 + a6b6 + a7b7 + a8b8 + a9b9

         '
fit <- sem(model, data = UKBMED)
sumfit=summary(fit)

In [ ]:
options(scipen = 999)

BFPMed=data.frame('var'=c('CRP','HT','DM','HDL','TG','Glc','HbA1c','DBP','SBP'),
                  'pval'=p.adjust(sumfit$PE[31:39,]$pvalue, method='BH'),
                  'est'=sumfit$PE[31:39,]$est,
                  'prop'=sumfit$PE[31:39,]$est/sumfit$PE[40,]$est
                 )

BFPMed